<a href="https://colab.research.google.com/github/tobypullan/tobypullan.github.io/blob/main/Micrograd_p3s1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building the neural net library
In this exercise, we will build classes for neurons, layers and a multi-layer perceptron (a basic neural network)

In [7]:
import random
import math

In [1]:
# Value class (taken from https://github.com/karpathy/nn-zero-to-hero/blob/master/lectures/micrograd/micrograd_lecture_second_half_roughly.ipynb)
class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out

  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out

  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')

    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward

    return out

  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')

    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward

    return out


  def backward(self):

    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [3]:
# Create a neuron class
class Neuron:
  """
  On initialisation, creates nin weights and a bias
  Each should be initialised to a Value with data between -1 and 1

  When called, should output the sum of each weight * its corresponding
  input and the bias, put through the tanh nonlinearity when it is called
  """
  def __init__(self, nin):
    pass

  def __call__(self, x):
    pass

In [4]:
# Create a layer class
class Layer:
  """
  On initalisation, create nout neurons that each have nin inputs
  Create nout neurons as each neuron produces 1 output

  When called, return the output of every neuron in the layer in a list
  """
  def __init__(self, nin, nout):
    pass
  def __call__(self, x):
    pass


In [5]:
# Create an MLP class
class MLP:
  """
  On initalisation, takes nin and nouts as parameters
  nouts is a list of ints containing the number of outputs of each layer
  nin is a single int
  Creates a layer for each of the nouts

  When called, return the output of the MLP by passing in x to the first layer
  then the output of the first layer into the second layer and so on
  """
  def __init__(self, nin, nouts):
    pass
  def __call__(self, x):
    pass

In [ ]:
# Test
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)